# Morph analyzer

In [1]:
import zipfile
import pandas as pd

from nltk.tokenize import RegexpTokenizer

Go to [OpenCorpora](http://opencorpora.org/dict.php) and download morph dict manually.

In [2]:
# with zipfile.ZipFile('dict.opcorpora.txt.zip', 'r').open('dict.opcorpora.txt') as f:
#     for s in f.readlines()[:100]:
#         print(s.decode('utf-8'))

In [3]:
%%time
simple_dict = {}
posses_zoo = set()
with zipfile.ZipFile('odict.csv.zip', 'r').open('odict.csv', 'r') as f:
    for line in f.readlines():
        splits = line.strip().decode('windows-1251').split(',')
        morphem, pos, paradigm = splits[0].strip(), splits[1].strip(), [word for word in splits[2:] if word.isalpha()]
        posses_zoo.add(pos)
        simple_dict[morphem] = (morphem, pos)
        for word in paradigm:
            simple_dict[word] = (morphem, pos)

CPU times: user 4.39 s, sys: 352 ms, total: 4.74 s
Wall time: 4.84 s


In [4]:
list(simple_dict.items())[:10]

[('а', ('а', 'част.')),
 ('Ааре', ('Ааре', 'ж')),
 ('аароновщина', ('аароновщина', 'ж')),
 ('аароновщины', ('аароновщина', 'ж')),
 ('аароновщине', ('аароновщина', 'ж')),
 ('аароновщину', ('аароновщина', 'ж')),
 ('аароновщиной', ('аароновщина', 'ж')),
 ('аароновщин', ('аароновщина', 'ж')),
 ('аароновщинам', ('аароновщина', 'ж')),
 ('аароновщинами', ('аароновщина', 'ж'))]

In [5]:
print(posses_zoo)

{'со', 'предик.', 'н', 'с', 'союз', 'нсв', 'св', 'числ.-п', 'жо', 'мо', 'мн.', 'сравн.', 'п', 'предл.', 'числ.', 'част.', 'мо-жо', 'межд.', 'м', 'св-нсв', 'вводн.', 'мс-п', 'ж'}


In [6]:
def translate_pos(pos):
    if pos in {'с', 'со', 'ж', 'жо', 'мн.', 'мо-жо', 'мо'}:
        return 'S'
    elif pos in {'п'}:
        return 'A'
    elif pos in {'св-нсв', 'св', 'нсв'}:
        return 'V'
    elif pos in {'предл.'}:
        return 'PR'
    elif pos in {'союз', 'част.'}:
        return 'CONJ'
    else:
        return 'ADV'

In [7]:
def analyze(text):
    def make_unit(word, morphem, pos):
        return word + '{' + morphem + '=' + pos + '}'
    
    tokenizer = RegexpTokenizer(r'\w+')
    parses = []
    for word in tokenizer.tokenize(text):
        if word in simple_dict:
            morphem, pos = simple_dict[word]
        elif word.lower() in simple_dict:
            morphem, pos = simple_dict[word.lower()]
        else:
            morphem, pos = word[:-1] + 'a', 'S'
        parses.append(make_unit(word, morphem, translate_pos(pos)))
    return ' '.join(parses)

## Validate

In [8]:
VALIDATE_TEXT = 'Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников. Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.'

In [9]:
analyze(VALIDATE_TEXT)

'Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S} Все{Всa=ADV} Гришины{Гришинa=ADV} одноклассники{одноклассник=S} уже{узкий=A} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=S} был{быa=ADV} чуть{чуть=CONJ} ли{ли=CONJ} не{не=CONJ} единственным{единственный=A} кого{когa=ADV} не{не=CONJ} вывозили{вывозить=V} никуда{никуда=ADV} дальше{дальше=ADV} Красной{красный=A} Пахры{Пахрa=ADV}'

## Test

In [10]:
with open('dataset_37845_1.txt', 'r') as i, \
    open('dataset_37845_1-a.txt', 'w') as o:
    for line in i:
        o.write(analyze(line.strip()) + '\n')